In [ ]:
import sympy as sm
import sympy.physics.mechanics as me
import time
import numpy as np
from scipy.integrate import solve_ivp
from scipy.optimize import fsolve, minimize
import matplotlib.pyplot as plt
import matplotlib as mp
from matplotlib.patches import Ellipse
%matplotlib inline
from IPython.display import HTML
mp.rcParams['animation.embed_limit'] = 2**128 # I need this for large animations on my iPad
from matplotlib import animation

Two homogeneous wheels of mass m and radius r are attached to a massless axle of length l and roll on an uneven street. The wheel is like a disc, no width. This means, the wheels are always perpenticular to the axis. This is used further down to locate $Dmc_L$ and also for the configuration constraint\
The wheels may rotate indenpendently of each other.

The basic shape of the street is modeled by strassen_form, the superimposed unevenness is 
modeled by strasse. (Strasse = Street in German, gesamt = combined in German), that is gesamt = strasse + strassen_form.

An 'observer', a particle of mass $m_o$, may be attached anywhere inside the wheel. 

=================================================================================================================================
Define the sympy symbols and the dynamic symbols needed for the calculation
- $q_L, q_R$ are the angle of the wheels w.r.t. AX.
- $u_L, u_R$ are the speeds
- $x_L, z_L, x_R z_R$ are the coordinates of the contact points 
- $u_{xL}, u_{zL}, u_{xR}, u_{zR}$  the speeds.


- the rest are constants needed to describe the physical properties of the system:
    - m is the mass of the wheels
    - $m_o$ is the mass of the observer
    - r is the diameter of the wheel
    - l is the distance of the wheel from each other, the length of the axle
    - $i_{XX}$, etc are the moments of inertial of the wheels
    - reibung is the friction
    - amplitude and frequenz determine the shape of the street
    - $ \alpha$ determines the location of the observer w.r.t. the center of the wheel. 
    
    
*length_config_constr* allows you to select the method of calculating the lemgth of the configuration constraint further down.\
( This is a 'left over' from a discussion a few months ago on Github, as to why $\bar v.normalize()$ may throw an error if $\bar v \approx 0$.
- 1: *config_constr.magnitude() is* used
- 2: *me.dot(config_constr, config_constr)* is used
- 3: *sm.sqrt(configX^2 + configY^2+ configZ^2)* is used, where configX := me.dot(config_constr, N.x), etc.

The printing of the information about the mass matrix and of the force vector takes time. By setting *information = False* this printing my be avoided.

This is a bit of a **monster**:\
With rempel = 1, the force vector has about 25 mio operations, the mass matrix has about 4.8 mio operations, running time on my iPad about 2 hours.\
With rempel = 2, the force vector has about 32 mio operations, the mass matrix has about 5.5 mio operations, running time on my iPad about 2.5 hours.\
With rempel = 3, the force vector has about 40 mio operations, the mass matrix has about 7.8 mio operations, running time on my iPad about 3 hours.\
With rempel = 4, the force vector has about 49 mio operations, the mass matrix has about 8.9 mio operations, running time on my iPad about 4 hours.

In [ ]:
start = time.time()
#========================
length_config_constr = 2
information = True
#========================

if length_config_constr not in [1, 2, 3] or isinstance(length_config_constr, int) == False:
    raise Exception('this must be an integer in (1, 2, 3)')

qL, qR, q2, q3 = me.dynamicsymbols('qL qR q2 q3', real=True) 
uL, uR, u2, u3 = me.dynamicsymbols('uL uR u2 u3', real=True)

xL, zL, xR, zR = me.dynamicsymbols('xL zL xR zR', real=True)           # coordinates of the contact points of the left / right wheel            
uxL, uzL, uxR, uzR = me.dynamicsymbols('uxL uzL uxR uzR', real=True)   # their 'speeds'


m, mo, g, r, l, iXX, iYY, iZZ = sm.symbols('m, mo, g, r, l, iXX, iYY, iZZ')
amplitude, frequenz, reibung, alpha = sm.symbols('amplitude frequenz reibung alpha')
t = me.dynamicsymbols._t

Define the frames and the points as follows:
- $N$ = inertial frame
- $A_X$ = frame attached to the axle and body fixed frame of the left wheel
- $A_R$ = frame attached to the right wheel
- $AX_h$ = auxiliary frame, needed for the rotational speed $u_R$ of the right wheel
- $P_0$ = fixed reference point
- $CP_L$ = contact point, where the left wheel touches the street.
- $CP_R$ =   dto for right wheel.
- $Dmc_L$ = center of the left wheel
- $Dmc_R$ = center of the right wheel
- $Po_L, Po_R$ = particles anywhwere within the wheel

At the bottom, I solve the linear equations for sm.Derivative(qL, t), etc.\
This is needed below, where these terms must be expressed as functions of $u_L$, $u_2$, etc.\
These equations are also used for the kinematic equations further down.

In [ ]:
N, AX, AXh, AR = sm.symbols('N, AX, AXh, AR', cls = me.ReferenceFrame)         
P0, CPL, CPR, DmcL, DmcR, PoL, PoR = sm.symbols('P0, CPL, CPR, DmcL, DmcR, PoL, PoR', cls = me.Point)

P0.set_vel(N, 0)

AX.orient_body_fixed(N, (q2, qL, q3), '213')
rot = AX.ang_vel_in(N)
AX.set_ang_vel(N, uL*AX.x + u2*AX.y + u3*AX.z)
rot1 = AX.ang_vel_in(N)

AXh.orient_axis(AX, -qL, AX.x)
AXh.set_ang_vel(AX, -uL * AX.x)

AR.orient_axis(AXh, qR, AXh.x)
rot11 = AR.ang_vel_in(N)
AR.set_ang_vel(AXh, uR * AXh.x)
rot12 = AR.ang_vel_in(N)

test = [me.dot(rot - rot1, uv) for uv in AXh]
print('Dynamic symbols of test', set().union(*[me.find_dynamicsymbols(test[i])for i in range(3)]), '\n')
test1, test2, test3 = sm.symbols('test1, test2, test3')
test_dict = {sm.Derivative(qL, t): test1, sm.Derivative(q2, t): test2, sm.Derivative(q3, t): test3}
test = [test[i].subs(test_dict) for i in range(3)]
antwort = sm.solve(test, (test1, test2, test3))

Here the street is modelled. The larger rumpel, the more 'uneven' the street will be. The smaller frequenz, the 'flatter' the street.
However larger rumple values will increase the force terms. rumpel must be an integer.

In [ ]:
#============================================
rumpel = 1                 
#============================================
if isinstance(rumpel, int) == False or rumpel < 1:
    raise Exception('rumpel must be an integer larger than zero')
    
def gesamt(x, z, amplitude, frequenz, rumpel):
    strasse = sum([amplitude/j * ((sm.sin(j*frequenz * x) + sm.sin(j*frequenz * z))) for j in range(1, rumpel)])
    strassen_form = (frequenz/2. * x)**2  + (frequenz/2. * z)**2
    return strasse + strassen_form 

This is to determine the maximum radius of the wheel so anywhere on the street it has only one contact point.
the radius of the wheel must be smaller than the smallest osculating circle (Schmiegekreis) of the road. 
Here I calculate this for the X and for the Z direction and simply take the smaller one of the two.\
(I did not bother to try and find the correct formula for higher dimensional functions)

In [ ]:
r_max_xL = (sm.S(1.) + (gesamt(xL, zL, amplitude, frequenz, rumpel).diff(xL))**2 )**sm.S(3/2)/gesamt(xL, zL, amplitude, frequenz, rumpel).diff(xL, 2)
r_max_zL = (sm.S(1.) + (gesamt(xL, zL, amplitude, frequenz, rumpel).diff(zL))**2 )**sm.S(3/2)/gesamt(xL, zL, amplitude, frequenz, rumpel).diff(zL, 2)

$CP_L$ and $CP_R$ are the contact points, where the wheels touch the street.

In [ ]:
CPL.set_pos(P0, xL*N.x + gesamt(xL, zL, amplitude, frequenz, rumpel)*N.y + zL*N.z)
CPR.set_pos(P0, xR*N.x + gesamt(xR, zR, amplitude, frequenz, rumpel)*N.y + zR*N.z)

**set the positions of** $Dmc_Lm Dmc_R, Po_L, Po_R$\
vektorL is the normal to the streel at point ($x_L, z_L$), the X/Z coordinates of the contact point $CP_L$. 
I found this formula in the internet.
It points 'inwards', hence the leading minus sign.
$Dmc_L$ is in the direction of this normal at distance *r* from $CP_L$. The wheel is 'thin', hence this vektor can be perpendicular to AX.x, that is, it has not component in AX.x direction. This is how I choose vektorL1.

In [ ]:
vektorL = -(gesamt(xL, zL, amplitude, frequenz, rumpel).diff(xL)*N.x - N.y + 
            gesamt(xL, zL, amplitude, frequenz, rumpel).diff(zL)*N.z)
vektorL1 = (me.dot(vektorL, AX.y) * AX.y + me.dot(vektorL, AX.z) * AX.z).simplify()
DmcL.set_pos(CPL, r * vektorL1.normalize())
DmcR.set_pos(DmcL, l * AXh.x)

PoL.set_pos(DmcL, r*alpha*AX.z)
PoR.set_pos(DmcR, r*alpha*AR.z)

**find the 'speed' of contact point $CP_L$**

The instantaneous contact point has no speed, as it is part of the street, too. We need $x(t), z(t)$ to get the location of subsequent contact points. These subsequent contact points will be located at $(x(t), gesamt(x(t), z(t)), z(t))$ .

Relationship of x(t) to q(t):
Obviously, $ x(t) = function(q(t), gesamt(x(t), z(t)), r) $.
When the ball is rotated through an angle $q$, the arc length is $r \cdot q(t)$.

The arc length of a function f(k(t)) from 0 to $x(t)$ is: $ \int_{0}^{x(t)} \sqrt{(1 +  \frac{d}{dx}(f(k(t)))^2)} \,dk \ $ (I found this in the internet)

This gives the sought after relationship between $q(t)$ and $x(t)$:
$ r \cdot (-q(t))  =  \int_{0}^{x(t)} \sqrt{(1 + \frac{d}{dk}(gesamt(k(t), z(t)))^2)} \,dk \ $, differentiated:
- $ r \cdot (-u)  = \sqrt{(1 + \frac{d}{dx}(gesamt(x(t), z(t)))^2)} \cdot \frac{d}{dt}(x(t)) $, that is solved for $\frac{d}{dt}(x(t))$:
- $ \frac{d}{dt}(x(t)) = \dfrac{-(r \cdot u)} {\sqrt{(1 + \frac{d}{dx}(gesamt(x(t), z(t)))^2)})} $

A similar formula holds for the speed in Z direction

$ \frac{d}{dt}(x(t)) = \dfrac{- (u3 \cdot r)} {\sqrt{(1 + \frac{d}{dx}(gesamt(x(t), z(t)))^2)}} $

$ \frac{d}{dt}(z(t)) =  \dfrac{(u1 \cdot r)} {\sqrt{(1 + \frac{d}{dz}(gesamt(x(t), z(t))))^2)}} $

As speeds are vectors, they can be added to give the resultant speed.

The + / - signs are a consequence of the 'right hand rule' for frames. These are the sought after first order differential equations for $(x(t), z(t))$.


In [ ]:
OMEGA = AX.ang_vel_in(N)
u3_wirk = me.dot(OMEGA, N.z)
u1_wirk = me.dot(OMEGA, N.x)
rhsx =  -u3_wirk * r / sm.sqrt(1. + (gesamt(xL, zL, amplitude, frequenz, rumpel).diff(xL))**2)
rhsz =   u1_wirk * r / sm.sqrt(1. + (gesamt(xL, zL, amplitude, frequenz, rumpel).diff(zL))**2)
print('rhsx DS: ', me.find_dynamicsymbols(rhsx))
print('rhsz DS: ', me.find_dynamicsymbols(rhsz))

**Set the speed of** $Dmc_L, Dmc_R, Po_L, Po_R$\
I use $v_{DmcL}$ = $D_{mcL}$.pos_from($P_0$).diff(t, N) to get the speed.\
*configdt_dict* is a dictionary needed here and also further down. It in turn needs the dictionary *antwort* caclulated above.\
For some reason, not clear to me, *v2pt_theory* did not give the correct results for $Po_L, Po_R$

In [ ]:
configdt_dict = {sm.Derivative(qL, t): antwort[test1], sm.Derivative(q2, t): antwort[test2], 
    sm.Derivative(q3, t): antwort[test3], sm.Derivative(xL, t): rhsx, sm.Derivative(zL, t): rhsz,
    sm.Derivative(xR, t): uxR, sm.Derivative(zR, t): uzR, uxL: rhsx, uzL: rhsz, sm.Derivative(qR, t): uR}

DmcL.set_vel(N, DmcL.pos_from(P0).diff(t, N).subs(configdt_dict))
DmcR.set_vel(N, DmcR.pos_from(P0).diff(t, N).subs(configdt_dict))

CPL.set_vel(N, CPL.pos_from(P0).diff(t, N).subs(configdt_dict))
CPR.set_vel(N, CPR.pos_from(P0).diff(t, N).subs(configdt_dict))
# for whatever reason v2pt_theory does not give the correct resilts
#PoL.v2pt_theory(DmcL, N, AX)
#PoR.v2pt_theory(DmcR, N, AR)
PoL.set_vel(N, PoL.pos_from(P0).diff(t, N).subs(configdt_dict))
PoR.set_vel(N, PoR.pos_from(P0).diff(t, N).subs(configdt_dict))

print('v(DmcL) DS viewed in N',me.find_dynamicsymbols(DmcL.vel(N), reference_frame=N))
print('v(DmcR) DS viewed in N',me.find_dynamicsymbols(DmcR.vel(N), reference_frame=N), '\n')
print('v(CPL)  DS viewed in N',me.find_dynamicsymbols(CPL.vel(N), reference_frame=N))
print('v(CPR)  DS viewed in N',me.find_dynamicsymbols(CPR.vel(N), reference_frame=N), '\n')
print('v(PoL)  DS viewed in N',me.find_dynamicsymbols(PoL.vel(N), reference_frame=N))
print('v(PoR)  DS viewed in N',me.find_dynamicsymbols(PoR.vel(N), reference_frame=N))

Set up the **configuration constraint for the axle**, get the corresponding **speed constraints**, and solve for $u_3, u_{xR}, u_{zR}$.\
The vector DmcR.pos_from(CPR) must not have a component in AX.x direction. The distance DmcR to CPR must be r.

Note: vektorL1 above and vektorR1 below are in the same plane, perpendicular to AX.x\
If in $vektor_{R1}$ the equivalent frame AXh (equivalent for the purpose here) is used, the number of operations in force shoot up.

In [ ]:
vektorR =  -(gesamt(xR, zR, amplitude, frequenz, rumpel).diff(xR)*N.x - N.y + 
             gesamt(xR, zR, amplitude, frequenz, rumpel).diff(zR)*N.z)
vektorR1 = (me.dot(vektorR, AX.y) * AX.y + me.dot(vektorR, AX.z) * AX.z).normalize()

config_constr = (DmcR.pos_from(CPR) - r * vektorR1)

print('config_constr DS', me.find_dynamicsymbols(config_constr, reference_frame=N),'\n')
print('config_constr FS', config_constr.free_symbols(reference_frame=N),'\n')
configX = me.dot(config_constr, N.x)
configY = me.dot(config_constr, N.y)
configZ = me.dot(config_constr, N.z)

if length_config_constr == 1:
    config_mag = config_constr.magnitude()
elif length_config_constr == 2:
    config_mag = me.dot(config_constr, config_constr) 
else:
    config_mag = sm.sqrt(configX**2 + configY**2 + configZ**2)

config_constr_matrix = sm.Matrix([configX, configY, configZ])  # needed to find the initial values of q3, xR, zR

configXdt = configX.diff(t).subs(configdt_dict)
configYdt = configY.diff(t).subs(configdt_dict)
configZdt = configZ.diff(t).subs(configdt_dict)

print('configXdt DS', me.find_dynamicsymbols(configXdt))
print('configXdt contains {} operations'.format(sum([configXdt.count_ops(visual=False) 
                                                     for i in range(3)])), '\n')
print('configYdt DS', me.find_dynamicsymbols(configYdt))
print('configYdt contains {} operations'.format(sum([configYdt.count_ops(visual=False) 
                                                     for i in range(3)])), '\n')
print('configZdt DS', me.find_dynamicsymbols(configZdt))
print('configZdt contains {} operations'.format(sum([configZdt.count_ops(visual=False) 
                                                     for i in range(3)])), '\n')

# Solve the linear equation as per Jason Moore's idea.
speed_mat = sm. Matrix([configXdt, configYdt, configZdt])
matrix_A = (speed_mat.jacobian([u3, uxR, uzR]))
vektor_b = (speed_mat.subs({i : 0. for i in [u3, uxR, uzR]}))
loesung1 = (matrix_A.LUsolve(-vektor_b))

print('loesung DS', me.find_dynamicsymbols(loesung1))
print('loesung FS', set.union(*[loesung1[i].free_symbols for i in range(3)]))
for i in range(len(loesung1)):
    print('loesung1 {} contains {} operations'.format(i, loesung1[i].count_ops(visual=False)))
loesung1 = [loesung1[0], loesung1[1], loesung1[2]]

**find $u_R$**


Obviously, $u_R$ is not independent.\
As $u_R$ is the rotation of the right wheel ($A_R$) around $AXh.x$, in can only be affected by the speed of $CP_R$ in the Z-direction.\
Above I found, now for the right wheel:


$ \frac{d}{dt}(z_R(t)) =  \dfrac{(u_R \cdot r)} {\sqrt{(1 + \frac{d}{dz_R}(gesamt(x_R(t), z_R(t))))^2)}} $

Solved for $u_R$:

$u_R$ = $\dfrac{1.}{r} \cdot \frac{d}{dt}(z_R(t)) \cdot \sqrt{(1 + \frac{d}{dz_R}(gesamt(x_R(t), z_R(t))))^2)}$

In [ ]:
vCPRz = me.dot(CPR.vel(N), AXh.z)

bedingung = (1. / r) * vCPRz * sm.sqrt(1. + (gesamt(xR, zR, amplitude, frequenz, rumpel).diff(zR))**2)

print('bedingung DS', me.find_dynamicsymbols(bedingung))
print('bedingung has {} operations'.format(bedingung.count_ops(visual=False)))

One has to solve for all four dependent generalized speeds, that is $u_3, u_{xR}, u_{zR}, u_R$ simultaneously. I will do it numerically later in the integration program. This is needed for the initial conditions. I also use it in the integration, following an idea by JM.

In [ ]:
speed_matrix = sm.Matrix([loesung1[0] - u3, loesung1[1] - uxR, loesung1[2]- uzR, bedingung - uR])

print('speed matrix DS',me.find_dynamicsymbols(speed_matrix))
print('speed_matrix has {} operations'.format(sum([speed_matrix[i].count_ops() for i in range(4)])))

The standard machinery to set up **Kane's equation** starts.

As the energy of a system may give hints that maybe a mistake was made setting up Kane's equations, e.g. absent any friction or applied forces/torques the total energy must be constant, I like to calculate them and look at them.

In [ ]:
IL = me.inertia(AX, iXX, iYY, iZZ)
IR = me.inertia(AR, iXX, iYY, iZZ)

BodyL = me.RigidBody('BodyL', DmcL, AX, m, (IL, DmcL))
BodyR = me.RigidBody('BodyR', DmcR, AR, m, (IR, DmcR))
PoLp  = me.Particle('PoLp', PoL, mo)
PoRp  = me.Particle('PoRp', PoR, mo)
BODY = [BodyL, BodyR, PoLp, PoRp]

punkte = [DmcL, DmcR, PoL, PoR]
massen = [m, m, mo, mo]
pot_energie = sum([masse*g * me.dot(punkt.pos_from(P0), N.y) for masse, punkt in zip(massen, punkte)])
kin_energie = sum([koerper.kinetic_energy(N).subs(configdt_dict) for koerper in BODY]) 

print('kinetic energy DS:', me.find_dynamicsymbols(kin_energie))
print('potential energy DS:', me.find_dynamicsymbols(pot_energie))

Set the **external forces** acting on the system.\
Here only the gravitational forces and possibly friction in the bearings of the wheels

In [ ]:
FL1 = [(DmcL, -m*g*N.y), (DmcR, -m*g*N.y), (PoL, -mo*g*N.y), (PoR, -mo*g*N.y)]
Torque = [(AX, -reibung * uL*AX.x), (AR, -reibung * uR*AX.x)  ]
FL = FL1 + Torque

It is very **important to use AX** when forming the kinematic equations of motion. Using N will increase the number of operations in force substantially. There is a discussion about this somewhere in GitHub, where Timo explains why this is so.

In [ ]:
start2 = time.time()
kd = [me.dot(rot - rot1, uv) for uv in AX] + [me.dot(rot11 - rot12, AXh.x)] + [
    uxR - xR.diff(t), uzR - zR.diff(t)]

speed_constraint = [uR - bedingung, u3 - loesung1[0], uxR - loesung1[1], uzR - loesung1[2]]
q_ind = [q2, q3, qL, qR] + [xR, zR]
u_ind = [u2, uL]
u_dep = [u3, uxR, uzR, uR]


KM = me.KanesMethod(N, q_ind=q_ind, u_ind=u_ind, u_dependent=u_dep, kd_eqs=kd, 
                    velocity_constraints=speed_constraint)
(fr, frstar) = KM.kanes_equations(BODY, FL)

MM1 = KM.mass_matrix_full
force1 = KM.forcing_full.subs(configdt_dict)

if information == True:
    print('force1 DS', me.find_dynamicsymbols(force1))
    print('force1 free symbols', force1.free_symbols)
    print('force1 has {} operations'.format(sum([force1[i].count_ops(visual=False) 
                for i in range(len(force1))])), '\n')
    print(MM1.shape)

print('it took {:.3f} sec to establish Kanes equations'.format(time.time() - start2))

Add $rhsx, rhsz$ at the bottom of force, to get $\dfrac{d}{dt}(x) = rhsx$, same for $z(t)$. 
This is to get $x_L(t), z_L(t)$ by numeric integration. 
  
- $ \dfrac{d}{dt}(x_L(t)) = rhsx $


- $ \dfrac{d}{dt}(z_L(t)) = rhsz $

I find it useful to print the dynamic symbols and the free symbols of such functions. This may give hints that something was wrong in describing the system. The number of operations is also useful to know, I think.

In [ ]:
force = sm.Matrix.vstack(force1, sm.Matrix([rhsx, rhsz]))
if information == True:
    print('force DS', me.find_dynamicsymbols(force))
    print('force free symbols', force.free_symbols)
    print('force has {} operations'.format(sum([force[i].count_ops(visual=False) 
                for i in range(len(force))])), '\n')

The mass matrix MM must be enlarged accordingly

In [ ]:
MM = sm.Matrix.hstack(MM1, sm.zeros(12, 2))
hilfs = sm.Matrix.hstack(sm.zeros(2, 12), sm.eye(2))
MM = sm.Matrix.vstack(MM, hilfs )
if information == True:
    print('MM DS', me.find_dynamicsymbols(MM))
    print('MM free symbols', MM.free_symbols)
    print('MM has {} operations'.format(sum([MM[i, j].count_ops(visual=False) 
                for i in range(MM.shape[0]) for j in range(MM.shape[1])])), '\n')

Convert all the sympy functions to numpy functions, so numerical calculations may be done.

In [ ]:
start2 = time.time()
pL = [m, mo, g, r, l, iXX, iYY, iZZ, amplitude, frequenz, reibung, alpha]
qL1 = q_ind + u_ind + u_dep + [xL, zL]
print('vector of variables is:', qL1)
MM_lam = sm.lambdify(qL1 + pL, MM, cse=True)
force_lam = sm.lambdify(qL1 + pL, force, cse=True)

print('it took {:.3f} sec to perform the lambdification'.format(time.time() - start2))

As it takes long to lambdify MM and force, I calculate the rest separately.

In [ ]:
start2 = time.time()

pot_lam = sm.lambdify(qL1 + pL, pot_energie, cse=True)
kin_lam = sm.lambdify(qL1 + pL, kin_energie, cse=True)

r_max_lam = sm.lambdify([xL, zL] + pL, [r_max_xL, r_max_zL], cse=True)

config_lam = sm.lambdify([q3, xR, zR] + [xL, zL, q2, qL] + pL, config_constr_matrix, cse=True)
config_mag_lam = sm.lambdify([q2, q3, qL, qR, xR, zR, xL, zL] + pL, config_mag, cse=True)
speed_lam = sm.lambdify([u3, uxR, uzR, uR] + q_ind + u_ind + [xL, zL] + pL, speed_matrix, cse=True )
speed_lam1 = sm.lambdify(qL1 + pL, speed_matrix, cse=True)

print('it took {:.3f} sec to perform the lambdification'.format(time.time() - start2))

Set the **parameters** and the **independent initial values**\
Their meaning is described above.\
I like to take names here which are similar to the ones used to get Kane"s equations, but the **same** names **must** be avoided. Otherwise, the symbols will be overwritten, with mostly uninteded consequences.\
If the street is almost flat, that is if $frequenz1 \approx 0.$ an error ('divide by zero') occurs. I am unsure why.

In [ ]:
#================================================================
m1 = 1.
mo1 = 1.e0
l1 = 4.
r1 = 2.

amplitude1 = 1.
frequenz1 = 0.25
reibung1 = 0.
alpha1 = 0.99

q21 = 0.000001
qL11, qR1 = 0.0, 0.0
xL1, zL1 = -5., 5.
u21, uL1 = -2., 3.

intervall = 25.
#================================================================
schritte = int(intervall * 300.)

iXX1 = 0.5 * m1 * r1**2   # from the internet
iYY1 = 0.5 * iXX1         # dto.
iZZ1 = 0.5 * iXX1         # dto.

print('Arguments:')
print('[m, mo, g, r, l, iXX, iYY, iZZ, amplitude, frequenz, reibung, alpha] = ')
pL_vals = [m1, mo1, 9.81, r1, l1, iXX1, iYY1, iZZ1, amplitude1, frequenz1, reibung1, alpha1]
print(pL_vals)

Find the initial values of the dependent variables and speeds:
- $q_3, x_R, z_R$
- $u_3, u_{xR}, u_{zR}, u_R$

In [ ]:
# find q3, xR, zR numerically, to satisfy the holonomic constraints.
def func(x0, args):  # this is only to get the arguments in a way to match fsolve / hol_mat_lam
    return config_lam(*x0, *args).reshape(3)

args = [xL1, zL1, q21, qL11] + pL_vals
x0 = tuple((1., 1., 1.))
for _ in range(3):
    q31, xR1, zR1 = fsolve(func, x0, args)
    x0 = tuple((q31, xR1, zR1))
fehler = config_mag_lam(q21, q31, qL11, qR1, xR1, zR1, xL1, zL1, *pL_vals)
print('error in initial configuration constraint: ', fehler)

#numerically find u3, uxR, uzR, uR
def func1(x0, *args):
    return speed_lam(*x0, *args).reshape(4)
x0 = tuple((0., 0., 0., 0.))
args = tuple([q21, q31, qL11, qR1, xR1, zR1] + [u21, uL1] + [xL1, zL1] + pL_vals)
u31, uxR1, uzR1, uR1 = fsolve(func1, x0, args)
print('error in initial speed constraints         ', [speed_lam(*[u31, uxR1, uzR1, uR1], 
        *[q21, q31, qL11, qR1, xR1, zR1], *[u21, uL1], *[xL1, zL1], *pL_vals)[i][0] for i in range(3)], '\n')

y0 = [q21, q31, qL11, qR1, xR1, zR1] + [u21, uL1] + [u31, uxR1, uzR1, uR1] + [xL1, zL1]
print('starting vector', [f'{j} = {i:.3f}' for i, j in zip(y0, 
        ('q21', 'q31', 'qL11', 'qR1', 'xR1', 'zR1', 'u21', 'uL1', 'u31', 
         'uxR1', 'uzR1', 'uR1', 'xL1', 'zL1'))])

The wheel must touch the street at exactly one point only. The radius of the wheel r must be smaller than the smallest bevel radius.
If the smallest bevel curve radius (Schmiegekreis) is smaller than the radius of the wheel, an exception is raised. 

In [ ]:
if frequenz1 != 0.:
    def func2(x, args):
# just needed to get the arguments matching for minimuze
        return np.abs(r_max_lam(*x, *args)[0])

    def func3(x, args):
# just needed to get the arguments matching for minimuze
        return np.abs(r_max_lam(*x, *args)[1])

    x0 = (0.1, 0.1)      # initial guess
    minimal1 = minimize(func2, x0, pL_vals)
    minimal2 = minimize(func3, x0, pL_vals)

    minimal = min(minimal1.get('fun'), minimal2.get('fun'))

    if pL_vals[3] < minimal:
        print('selected radius = {} is less than minimally admissible radius = {:.4f}, hence o.k.'
          .format(pL_vals[3], minimal), '\n')
    else:
        print('selected radius {} is larger than admissible radius {:.4f}, hence NOT o.k.'
          .format(pL_vals[3], minimal), '\n')
        raise Exception('Radius of wheel is too large')
else:
    print('the street is flat')

**Numerical integration starts.**

Following an idea of Jason Moore, I numerically calculate the dependent generalized coordinates and so *force* them to be closer to the *correct* value. I loop over the numerical solution a few times, as this seems to improve the accuracy of the solution. I do the same with the speed constraints, seems to help.

In [ ]:
start1 = time.time()
times = np.linspace(0, intervall, schritte)
x0 = [q31, xR1, zR1]           # initial guesses
x01 = [u31, uxR1, uzR1, uR1]   # dto.

def gradient(t, y, args):
    if time.time() - start1 > 300:
        print('abortion time:', t)
        raise Exception('running too long')
    global x0, x01
    args1 = [y[-2], y[-1], y[0], y[2]] + args
    for _ in range(3):
        y[1], y[4], y[5] = fsolve(func, x0, args1)
        x0 = [y[1], y[4], y[5]]  # improved guess
        
        args2 = tuple([y[k] for k in range(8)] + [y[-2], y[-1]] + args)
        y[8], y[9], y[10], y[11] = fsolve(func1, x01, args2)
        x01 = [y[8], y[9], y[10], y[11]]   # improved guess
    
    vals = np.concatenate((y, args))
    sol = np.linalg.solve(MM_lam(*vals), force_lam(*vals))
    return np.array(sol).T[0]

t_span = (0., intervall)
resultat1 = solve_ivp(gradient, t_span, y0, t_eval=times, args=(pL_vals,), atol=1.e-8
    , rtol=1.e-8, method='BDF')

resultat = resultat1.y.T
print(resultat.shape)
event_dict = {-1: 'Integration failed', 0: 'Integration finished successfully', 1: 'some termination event'}
print(event_dict[resultat1.status], 'message was:', resultat1.message)
    
print('resultat shape', resultat.shape, '\n')

print("To numerically integrate an intervall of {} sec, the routine cycled {} times and it took {:.3f} sec "
      .format(intervall, resultat1.nfev, time.time() - start1))

Plot the **deviations of the configuration constraint** and of the resulting **speed constraints.**

In [ ]:
schritte1 = resultat.shape[0]
times1 = [times[i] for i in range(schritte1)]
config_np = np.empty(schritte1)
zaehler = 0
for i in range(schritte1):
    config_np[i] = config_mag_lam(*[resultat[i, j] for j in range(6)], resultat[i, -2], 
        resultat[i, -1], *pL_vals)
    if config_np[i] < 0.:
        zaehler += 1
minimum = np.min(config_np)
print('There are {} times, when dot(config, config) < 0. Mimimum value was {}'.format(zaehler, minimum))
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(times1, config_np)
ax.set_title('Violation of configuration constraint')
label_dict = {1: 'config_constr.magnitude', 2: 'me.dot(config_constr, config_constr)', 
             3: ' sm.sqrt(configX**2 + configY**2 + configZ**2)'}
ax.set_xlabel("time (sec)")
ax.set_ylabel('method used to get config_mag is: \n' + label_dict[length_config_constr])

speed_np = np.empty((schritte1, 4))
for i in range(schritte1):
    speed_np[i] = speed_lam1(*[resultat[i, j] for j in range(resultat.shape[1])], *pL_vals).reshape(4)
    
fig, ax = plt.subplots(figsize=(10, 5))
bezeichnung = ['u3', 'uxR', 'uzR', 'uR']
for i in range(4):
    ax.plot(times1, speed_np[:, i], label=bezeichnung[i])
ax.set_title('violation of speed constraints')
ax.set_xlabel('time (sec)')
ax.legend();

Plot any generalized coordinates / speeds you want to see.

In [ ]:
bezeichnung = ['q21', 'q31'] + ['qL1', 'qR1'] + ['xR1', 'zR1'] + ['u21', 'uL1'] + ['u31', 'uxR1', 'uzR1', 
               'uR1'] + ['xL1', 'zL1']
fig, ax = plt.subplots(figsize=(10, 5))
for i in (7, 11):      #(resultat.shape[1]):
    ax.plot(times[: resultat.shape[0]], resultat[:, i], label = bezeichnung[i])
ax.set_title('generalized coordinates')
ax.set_xlabel('time (sec)')
ax.legend();

**Plot the energies of the system**\
Total energy must be constant, absent any friction.

In [ ]:
pot_np = np.empty(resultat.shape[0])
kin_np = np.empty(resultat.shape[0])

total_np = np.empty(resultat.shape[0])
for k1 in range(resultat.shape[0]):
    pot_np[k1]    = pot_lam(*[resultat[k1, j] for j in range(resultat.shape[1])], *pL_vals)
    kin_np[k1]    = kin_lam(*[resultat[k1, j] for j in range(resultat.shape[1])], *pL_vals)
    total_np[k1] = pot_np[k1] + kin_np[k1]

if reibung1 == 0.:
    fehler = ((xyz:=max(total_np)) - min(total_np)) / xyz * 100.
    print('max deviation from total energy = constant is {:.3f} % of max total energy'.format(fehler))
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(times[: resultat.shape[0]], kin_np, label = 'kin energy')
ax.plot(times[: resultat.shape[0]], pot_np, label = 'pot energy')
#ax.plot(times[: resultat.shape[0]], spring_np, label = 'spring energy')
ax.plot(times[: resultat.shape[0]], total_np, label = 'total energy')
ax.set_title('Energy of the system, friction = {}'.format(reibung1))
ax.set_xlabel('time (sec)')
ax.legend();


This gives the movement of the system, projected on the X / Z horizontal plane. The height above the lowest point of the street is indicated by its color.\
When the axle is not parallel to the X/Z plane, the projection of the discs are ellipses. The axle 'gets shorter'.\
As HTML is so slow, I restrict the points in time to be considered to around *zeitpunkte*.

In [ ]:
#================
zeitpunkte = 500
#================

DmcL_pos = [(me.dot(DmcL.pos_from(P0), uv)) for uv in (N.x, N.y, N.z)]
DmcR_pos = [(me.dot(DmcR.pos_from(P0), uv)) for uv in (N.x, N.y, N.z)]
PoL_pos  = [(me.dot(PoL.pos_from(P0), uv)) for uv in (N.x, N.z)]
PoR_pos  = [(me.dot(PoR.pos_from(P0), uv)) for uv in (N.x, N.z)]

DmcL_lam = sm.lambdify(qL1 + pL, DmcL_pos, cse=True)
DmcR_lam = sm.lambdify(qL1 + pL, DmcR_pos, cse=True)
PoL_lam  = sm.lambdify(qL1 + pL, PoL_pos, cse=True)
PoR_lam  = sm.lambdify(qL1 + pL, PoR_pos, cse=True)


# auxiliary points for the wheels
richtung1 = DmcR.pos_from(DmcL).cross(N.y)
richtung  = richtung1.normalize()

richtungE = (me.dot(richtung1, N.x) * N.x - me.dot(richtung, N.z) * N.z)


# angle for the direction of the ellipses.
hilfs1 = me.dot(richtung1, N.x)
angle1 = sm.atan(me.dot(richtung1, N.z) / hilfs1)
angle1_lam = sm.lambdify(qL1 + pL, angle1, cse=True)

# calculate the angle the wheels are tilted. winkel actually is sinus(angle), which is needed.
komp       = [me.dot(DmcR.pos_from(DmcL), uv) for uv in N]
hypothe    = DmcR.pos_from(DmcL).magnitude()
winkel     = komp[1] / hypothe
winkel_lam = sm.lambdify(qL1 + pL, winkel, cse=True)

resultat2 = []
times2    = []
reduction = int(resultat.shape[0]/zeitpunkte)
for l11 in range(resultat.shape[0]):
    if l11 % reduction == 0:
        resultat2.append(resultat[l11])
        times2.append(times[l11])
schritte2  = len(times2)
resultat2  = np.array(resultat2)
times2     = np.array(times2)
print(f'number of time points considered is {schritte2}')

DmcL_np = np.empty((schritte2, 3))
DmcR_np = np.empty((schritte2, 3))
PoL_np  = np.empty((schritte2, 2))
PoR_np  = np.empty((schritte2, 2))

winkel_np = np.empty(schritte2)
angle1_np = np.empty(schritte2)

for l11 in range(schritte2):
    DmcL_np[l11]     = DmcL_lam(*[resultat2[l11, j] for j in range(resultat2.shape[1])], *pL_vals)
    DmcR_np[l11]     = DmcR_lam(*[resultat2[l11, j] for j in range(resultat2.shape[1])], *pL_vals)
    PoL_np[l11]      =  PoL_lam(*[resultat2[l11, j] for j in range(resultat2.shape[1])], *pL_vals)
    PoR_np[l11]      =  PoR_lam(*[resultat2[l11, j] for j in range(resultat2.shape[1])], *pL_vals)
        
    winkel_np[l11] = winkel_lam(*[resultat2[l11, j] for j in range(resultat2.shape[1])], *pL_vals)
    angle1_np[l11] = np.rad2deg(angle1_lam(*[resultat2[l11, j] for j in range(resultat2.shape[1])], 
       *pL_vals))
    
# get the size of the picture
xmin = min(np.min(DmcL_np[:, 0]), np.min(DmcR_np[:, 0]), np.min(PoL_np[:, 0]), np.min(PoR_np[:, 0]), 
           np.min(PoL_np[:, 0]), np.min(PoR_np[:, 0]))    
xmax = max(np.max(DmcL_np[:, 0]), np.max(DmcR_np[:, 0]), np.max(PoL_np[:, 0]), np.max(PoR_np[:, 0]), 
           np.max(PoL_np[:, 0]), np.max(PoR_np[:, 0]))

ymin = min(np.min(DmcL_np[:, 2]), np.min(DmcR_np[:, 2]), np.min(PoL_np[:, 1]), np.min(PoR_np[:, 1]), 
           np.min(PoL_np[:, 1]), np.min(PoR_np[:, 1]))    
ymax = max(np.max(DmcL_np[:, 2]), np.max(DmcR_np[:, 2]), np.max(PoL_np[:, 1]), np.max(PoR_np[:, 1]), 
           np.max(PoL_np[:, 1]), np.max(PoR_np[:, 1]))

maximum = max(np.abs(xmin), np.abs(xmax), np.abs(ymin), np.abs(ymax))

DmcLF = np.empty(schritte2, dtype=int)
DmcRF = np.empty(schritte2, dtype=int)

# the color mappings below require integers. As the height variation of the mass centers of the wheels
# is not much, I multiply by a factor of 500 to get a decent number of different colors.
faktor = 1000.
for i in range(schritte2):
    DmcLF[i] = int(DmcL_np[i, 1] * faktor)
    DmcRF[i] = int(DmcR_np[i, 1] * faktor)

farbe_max = max(np.max(DmcLF), np.max(DmcRF))
farbe_min = min(np.min(DmcLF), np.min(DmcRF))

Test = mp.colors.Normalize(farbe_min, farbe_max)
Farbe = mp.cm.ScalarMappable(Test, cmap='plasma')
farbeL1 = Farbe.to_rgba(DmcLF[0])    # color of the starting position of left center of wheel
farbeR1 = Farbe.to_rgba(DmcRF[0])    # color of the starting position of left center of wheel

def animate_pendulum(times, DmcL_np, DmcR_np, PoL_np, PoR_np, DmcLF, DmcRF, winkel_np):
    
    fig, ax = plt.subplots(figsize=(8, 8), subplot_kw={'aspect': 'equal'})
    fig.colorbar(Farbe, label='Height of the ball above ground level\n times {}'.format(faktor), 
                 shrink=0.9, ax=ax)
    ax.axis('on')
    ax.set_xlim(-maximum - 1., maximum + 1.)
    ax.set_ylim(-maximum - 1., maximum + 1.)
    ax.set_xlabel(' X Axis', fontsize=15)
    ax.set_ylabel('Z Axis', fontsize=15)

    line1, = ax.plot([], [], 'o', markersize=1, color=farbeL1)      # DmcL
    line1a,= ax.plot([], [], linestyle='-', color='red', linewidth=0.5) # trace DmcL
    line2, = ax.plot([], [], 'o', markersize=1, color=farbeR1)      # DmcR
    line2a,= ax.plot([], [], linestyle='-', color='blue', linewidth=0.5)  # trace DmcR 
    line3, = ax.plot([], [], 'o', markersize=5, color='red')         # PoL
    line4, = ax.plot([], [], 'o', markersize=5, color='green')       # PoR
    line7, = ax.plot([], [], 'bo', markersize=1, linestyle='-')      # axle
    
    elliL = Ellipse((DmcL_np[0, 0], DmcL_np[0, 2]), width=2.*r1, height=2.*winkel_np[0]*r1, 
            angle=angle1_np[0], fill=True, color=farbeL1, ec=farbeL1, alpha=0.5, linewidth=3)
    ax.add_patch(elliL)
    
    elliR = Ellipse((DmcR_np[0, 0], DmcR_np[0, 2]), width=2.*r1, height=2.*winkel_np[0]*r1, 
            angle=angle1_np[0], fill=True, color=farbeR1, ec=farbeR1,alpha=0.5, linewidth=3)
    ax.add_patch(elliR)
    
    
    def animate(i):
        farbeLi = Farbe.to_rgba(DmcLF[i])                            # color of the actual point at time i
        farbeRi = Farbe.to_rgba(DmcRF[i])                            # color of the actual point at time i
        ax.set_title('running time {:.1f} sec'.format(times[i]), fontsize=15)
        line1.set_data(DmcL_np[i, 0], DmcL_np[i, 2])
        line1.set_color(farbeLi)
        line1a.set_data(DmcL_np[: i, 0], DmcL_np[: i, 2])
        line2.set_data(DmcR_np[i, 0], DmcR_np[i, 2])
        line2.set_color(farbeRi)
        line2a.set_data(DmcR_np[: i, 0], DmcR_np[: i, 2])
        line3.set_data(PoL_np[i, 0], PoL_np[i, 1])
        line4.set_data(PoR_np[i, 0], PoR_np[i, 1])
        
        x_werte = [DmcL_np[i, 0], DmcR_np[i, 0]]
        y_werte = [DmcL_np[i, 2], DmcR_np[i, 2]]
        line7.set_data(x_werte, y_werte)
        
        elliL.set_center((DmcL_np[i, 0], DmcL_np[i, 2]))
        elliL.set_height(2.*winkel_np[i]*r1)
        elliL.set_angle(angle1_np[i])
        elliL.set_color(farbeLi)
        
        elliR.set_center((DmcR_np[i, 0], DmcR_np[i, 2]))
        elliR.set_height(2.*winkel_np[i]*r1)
        elliR.set_angle(angle1_np[i])
        elliR.set_color(farbeRi)
        
        return line1, line1a, line2, line2a, line3, line4, line7, elliL, elliR,

    anim = animation.FuncAnimation(fig, animate, frames=len(times2),
                                   interval=1000*max(times2) / len(times2),
                                   blit=True)
    plt.close(fig)
    return anim

anim = animate_pendulum(times2, DmcL_np, DmcR_np, PoL_np, PoR_np, DmcLF, DmcRF, winkel_np)

HTML(anim.to_jshtml())    